<a href="https://colab.research.google.com/github/Annmodels/mnist_recon/blob/master/CNN_details.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from numpy import asarray
import cv2
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
from google.colab import files
files.upload()

In [0]:
img = cv2.imread('Lenna.png',cv2.IMREAD_GRAYSCALE)/255
plt.imshow(img, cmap = 'gray')
plt.show()
img.shape
           

In [0]:
class Conv:
  def __init__(self, num_filters , filter_size):
    self.num_filters = num_filters
    self.filter_size = filter_size
    self.conv_filter = np.random.randn(num_filters,filter_size,filter_size)/(filter_size*filter_size)
  
  def image_region(self,image):
    height,width = image.shape
    self.image = image
    for j in range(height - self.filter_size + 1):
      for k in range(width - self.filter_size + 1):
        image_patch = image[j:(j+self.filter_size),k:(k+self.filter_size)]
        yield image_patch,j,k
  
  def forward_prop(self,image):
    height,width = image.shape
    conv_out = np.zeros((height - self.filter_size + 1 , width - self.filter_size + 1 , self.num_filters))
    for image_patch,i,j in self.image_region(image):
      conv_out[i,j] = np.sum(image_patch*self.conv_filter , axis = (1,2))
    return conv_out
  
  def back_prop(self,dl_dout,learning_rate):
    dl_df_params = np.zeros(self.conv_filter.shape)
    for image_patch,i,j in self.image_region(self.image):
      for k in range(self.num_filters):
        dl_df_params[k] += image_patch*dl_dout[i,j,k]
      
    self.conv_filter -= learning_rate*dl_df_params
    return dl_df_params

convrun = Conv(18,7)
out = convrun.forward_prop(img)
out.shape
plt.imshow(out[:,:,17] , cmap = 'gray')
plt.show()

out.shape
                     

In [0]:
class MaxPool:
  def __init__(self , filter_size):
    self.filter_size = filter_size
   # print(self.filter_size)
  
  def image_region(self,image):
    #print(self.filter_size)
    new_height = image.shape[0] // self.filter_size
    new_width = image.shape[1] // self.filter_size
    self.image = image
    for i in range(new_height):
      for j in range(new_width):
        image_patch = image[(i*self.filter_size):(i*self.filter_size + self.filter_size),(j*self.filter_size):(j*self.filter_size + self.filter_size)]
        yield image_patch,i,j

  def forward_prop(self,image):
    height,width,num_filters = image.shape
   # print(height,width,num_filters)
    output = np.zeros((height//self.filter_size,width//self.filter_size,num_filters))
   # print(output.shape)
    for image_patch,i,j in self.image_region(image):
      #print("hi")
     # print(image_patch.shape)
      output[i,j] = np.amax(image_patch,axis = (0,1))
    return output

  def back_prop(self,dl_out):
    dl_dmax_pool = np.zeros(self.image.shape)
    for image_patch,i,j in self.image_region(self.image):
      height,width,num_filters = image_patch.shape
      max_val = np.amax(image_patch , axis = (0,1))
      for il in range(height):
        for jl in range(width):
          for kl in range(num_filters):
            if image_patch[il,jl,kl] == max_val[kl]:
              dl_dmax_pool[i*self.filter_size+il , j*self.filter_size+jl , kl] == dl_out[i,j,kl]
      return dl_dmax_pool

#print(out.shape)
convrun2 = MaxPool(4)
out2 = convrun2.forward_prop(out)
out2.shape
plt.imshow(out2[:,:,17] , cmap = 'gray')
plt.show()       
    
out2.shape


In [0]:
class Softmax:
  def __init__(self,input_node,softmax_node):
    self.weight = np.random.rand(input_node,softmax_node)/input_node
    self.bias = np.zeros(softmax_node)

  def forward_prop(self,image):
    self.orig_im_shape = image.shape
    image_modified = image.flatten()
    self.modified_input = image_modified
    output_val = np.dot(image_modified,self.weight) + self.bias
    self.out = output_val
    exp_out = np.exp(output_val)
    return exp_out/np.sum(exp_out,axis = 0)

  def back_prop(self,dl_dout,learning_rate):
    for i,grad in enumerate(dl_dout):
      if grad == 0:
        continue
      transformation_eq = np.exp(self.out)
      s_total = np.sum(transformation_eq)
      #Gradients wrt out(z)
      dy_dz = -transformation_eq[i]*transformation_eq/(s_total**2)
      dy_dz[i] = transformation_eq[i]*(s_total-transformation_eq[i])/(s_total**2)
      #Gradients of totals wrt wtnb and  input
      dz_dw = self.modified_input
      dz_db = 1
      dz_d_inp = self.weight
      #Gradient of loss wrt totals
      dl_dz = grad*dy_dz
      #Gradients of loss against wtnb input
      dl_dw = dz_dw[np.newaxis].T@dl_dz[np.newaxis]
      dl_db = dl_dz*dz_db
      dl_d_inp = dz_d_inp@dl_dz
      self.weight -= learning_rate*dl_dw
      self.bias -= learning_rate*dl_db
      return dl_d_inp.reshape(self.orig_im_shape)

convrun3 = Softmax(126*126*18,10)
out3 = convrun3.forward_prop(out2)
print(out3)
                                


In [0]:
from keras.datasets import mnist
(xtrain,ytrain),(xtest,ytest) = mnist.load_data()
train_images = xtrain[:1500]
train_labels = ytrain[:1500]
test_images = xtest[:1500]
test_labels = ytest[:1500]

conv = Conv(8,3)
pool = MaxPool(2)
softmax = Softmax(13*13*8,10)

def cnn_forward_prop(image,label):
  out_p = conv.forward_prop((image/255) - 0.5)
  out_p = pool.forward_prop(out_p)
  out_p = softmax.forward_prop(out_p)
  cross_ent_loss = -np.log(out_p[label])
  acc_eval = 1 if np.argmax(out_p) == label else 0
  return out_p,cross_ent_loss,acc_eval

def training_cnn(image,label,learn_rate = 0.005):
  out,loss,acc = cnn_forward_prop(image,label)
  gradient = np.zeros(10)
  gradient[label] = -1/out[label]
 # print("hi")
  #print(gradient[label])

  #backprop
  grad_back = softmax.back_prop(gradient,learn_rate)
  grad_back = pool.back_prop(grad_back)
  grad_back = conv.back_prop(grad_back,learn_rate)
  return loss,acc

for epoch in range(4):
  print('epoch %d' %(epoch+1))
  shuffle_data = np.random.permutation(len(train_images))
  train_images = train_images[shuffle_data]
  train_labels = train_labels[shuffle_data]
  loss = 0
  num_correct = 0
  for i,(im,label) in enumerate(zip(train_images,train_labels)):
    if i%100 == 0:
      print('%d steps: avg_loss %.3f acc %d percent'%(i+1,loss/100,num_correct))
      loss = 0
      num_correct = 0
    l1,acc = training_cnn(im,label)
    #print("l1")
   # print(l1)
    loss += l1
    num_correct += acc

    

In [0]:
print("Testing phase")
loss = 0
num_correct = 0
for im,label in zip(test_images,test_labels):
  _,l1,acc = cnn_forward_prop(im,label)
  loss += l1
  num_correct += acc
num_tests = len(test_images)
#print("Test loss:",loss/num_tests)
print("Test acc:",num_correct/num_tests)
